In [190]:
import pandas as pd
import torch

data = pd.read_csv('data.csv')

In [191]:
one_hot = pd.get_dummies(data['Intent'])
data = data.drop('Intent', axis=1)
data = data.join(one_hot)

data.head()

,Example,Complaint,Farewell,Feedback,Greet,Inquiry,Navigation,Request
0,Hi,False,False,False,True,False,False,False
1,Hello,False,False,False,True,False,False,False
2,Hey there,False,False,False,True,False,False,False
3,Good morning,False,False,False,True,False,False,False
4,Howdy,False,False,False,True,False,False,False


In [192]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, train_size=0.8, random_state=10)
validation_data, test_data = train_test_split(test_data, train_size=0.5, random_state=10)

train_data.shape, test_data.shape, validation_data.shape

((292, 8), (37, 8), (36, 8))

In [193]:
# print the first 5 rows of the train_data
train_data.head()

,Example,Complaint,Farewell,Feedback,Greet,Inquiry,Navigation,Request
34,A warm welcome to class!,False,False,False,True,False,False,False
87,Until we meet again,False,True,False,False,False,False,False
57,Have a good one,False,True,False,False,False,False,False
152,How do I change my mailing address?,False,False,False,False,True,False,False
204,The product is defective.,True,False,False,False,False,False,False


In [194]:
# turn everything to lowercase
train_data['Example'] = train_data['Example'].str.lower()
test_data['Example'] = test_data['Example'].str.lower()
validation_data['Example'] = validation_data['Example'].str.lower()

In [195]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokens = tokenizer.tokenize(train_data['Example'].values[0])

tokenizer.convert_tokens_to_ids(tokens)

/Users/lakshithnishshanke/miniconda3/envs/labml/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[1037, 4010, 6160, 2000, 2465, 999]

In [196]:
from torch.utils.data import Dataset

all_letters = 'abcdefghijklmnopqrstuvwxyz\'?! '
n_letters = len(all_letters)

def letterToIndex(letter):
    return all_letters.find(letter)

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters, dtype=torch.float32)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

def lineToTensor(line):
    tensor = torch.zeros(80, 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

class CustomDataset(Dataset):
    def __init__(self, data):
        sentences = data['Example'].values
        
        print(len(sentences))
        
        inputs = [lineToTensor(sentence) for sentence in sentences]
        
        self.inputs = torch.cat(inputs).reshape(len(inputs), inputs[0].shape[0], n_letters)
            
        print(self.inputs.shape)
        
        self.outputs = torch.tensor(data.drop('Example', axis=1).values, dtype=torch.float32)

    def __getitem__(self, index):
        return self.inputs[index], self.outputs[index]

    def __len__(self):
        return len(self.inputs)

In [197]:
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)
validation_dataset = CustomDataset(validation_data)

292
torch.Size([292, 80, 30])
37
torch.Size([37, 80, 30])
36
torch.Size([36, 80, 30])


In [198]:
batch_size = 16

In [199]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

len(train_loader), len(test_loader), len(validation_loader)

(19, 3, 3)

In [200]:
# print the first batch of the train_loader
next(iter(train_loader))

[tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 1.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 1.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 1.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [201]:
import torch.nn as nn
import torch.nn.functional as F

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.hidden_size = 64

        self.i2h = nn.Linear(30, self.hidden_size)
        self.h2h = nn.Linear(self.hidden_size, self.hidden_size)
        self.h2o = nn.Linear(self.hidden_size, 7)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(30, self.hidden_size)

n_hidden = 64
rnn = RNN()

rnn = rnn.float()

In [202]:
# input = torch.tensor([1.0])
# hidden = torch.zeros(1, 64)
# 
# output, next_hidden = rnn(input, hidden)
# output

In [203]:
criterion = nn.NLLLoss()
learning_rate = 1e-4

In [204]:
import wandb

configs = {
    "learning_rate": learning_rate,
    "architecture": "RNN",
    "dataset": "custom",
}

wandb.init(
    project="Intellihack_2",
    config=configs
)

Accuracy,▃▁▁▁▁▁▁▁▁▁▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆████████████
Loss,▅▃▅▄▄█▄▁▅▄▅▃▄▄▄▄▅▃▆▃▂▂▅▂▅▅▃▅▄▃▅▂▃▄▅▂▃▁▂▅
Accuracy,0.24324
Loss,1.90028


In [208]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    line_tensor = line_tensor.float()
    
    # print(torch.unsqueeze(line_tensor[:,0], 1).shape)

    rnn.zero_grad()

    for i in range(line_tensor.size(1)):
        output, hidden = rnn(torch.unsqueeze(line_tensor[:,i], 1), hidden)
    
    print(output.shape, category_tensor.shape)
    
    # convert category tensor to indices
    category_tensor = torch.argmax(category_tensor, dim=1)
    
    loss = criterion(output, category_tensor)
    loss.backward()

    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    wandb.log({
        "Loss": loss.item()
    })

    print(loss.item())

In [209]:
def test_evaluate():
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (input, output_data) in enumerate(test_loader):
            hidden = rnn.initHidden()
            input = input.float()
            
            for i in range(input.size(1)):
                output, hidden = rnn(torch.unsqueeze(input[:,i], 1), hidden)
            
            total += output.size(0)
            correct += (torch.argmax(output_data, dim=1) == torch.argmax(output, dim=1)).sum().item()
            
        wandb.log({
            "Accuracy": correct / total
        })
            
    print('Accuracy of the network on the test examples: %d %%' % (100 * correct / total))

In [210]:
# iterate data loader and train
epochs = 5000
for epoch in range(epochs):
    for i, (input, output) in enumerate(train_loader):
        train(output, input)
    test_evaluate()

torch.Size([16, 30, 7]) torch.Size([16, 7])


RuntimeError: Expected target size [16, 7], got [16]